In [20]:
import numpy as np


def lennard_jones_potential(distance):
    if distance == 0:
        return float("-inf")
    return 1 / distance**12 - 2 / distance**6


def calculate_total_energy(positions):
    energy_sum = 0
    num_atoms = len(positions) // 3
    positions_with_origin = np.concatenate((positions, [0, 0, 0]))
    position_matrix = positions_with_origin.reshape((num_atoms + 1, 3))

    for i in range(num_atoms + 1):
        for j in range(i + 1, num_atoms + 1):
            energy_sum += lennard_jones_potential(
                np.linalg.norm(position_matrix[i] - position_matrix[j])
            )
    return energy_sum


def calculate_gradients(positions):
    num_atoms = len(positions) // 3
    gradients = np.zeros_like(positions)

    for i in range(num_atoms):
        current_pos = positions[3 * i : 3 * i + 3]
        gradient_i = 12 * current_pos / (
            np.linalg.norm(current_pos) ** 7
        ) - 12 * current_pos / (np.linalg.norm(current_pos) ** 4)

        for j in range(num_atoms):
            if i == j:
                continue
            other_pos = positions[3 * j : 3 * j + 3]
            r_vec = current_pos - other_pos
            r_norm = np.linalg.norm(r_vec) ** 7
            gradient_i += 12 * r_vec / r_norm

        gradients[3 * i : 3 * i + 3] = gradient_i

    return gradients


def wolfe_condition(step_size, c1, pos, grad):
    next_pos = pos - step_size * grad
    next_energy = calculate_total_energy(next_pos)
    current_energy = calculate_total_energy(pos)

    condition = next_energy <= current_energy + c1 * step_size * np.dot(grad, -grad)
    return condition


def perform_line_search(
    positions, gradient, initial_step_size=0.01, c1=0.2, shrinkage=0.5
):
    step_size = initial_step_size

    while not wolfe_condition(step_size, c1, positions, gradient):
        step_size *= shrinkage

    return step_size


def gradient_descent(positions, learning_rate=0.001, max_iterations=1000):
    history = []

    for _ in range(max_iterations):
        gradients = calculate_gradients(positions)
        step_size = perform_line_search(positions, gradients)
        positions -= step_size * gradients
        history.append(positions.copy())

        if np.linalg.norm(gradients) < 1e-10:
            break

    return positions, history

In [21]:
N = 3
init = np.random.rand(3*(N-1))   

optimum, history = gradient_descent(init)

print("Optimized positions:", optimum)
print("Final energy:", calculate_total_energy(optimum))

Optimized positions: [ -1.27879788  19.58242947   1.01606987   1.59158712 -18.82309773
   0.40952589]
Final energy: -7.930301675361927e-08
